## FAST-TEXT

In [161]:
from fasttext import train_supervised 
import pandas as pd
import csv
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [162]:
df_train_neg = pd.read_csv("data/twitter-datasets/train_neg.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
df_train_pos = pd.read_csv("data/twitter-datasets/train_pos.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
df_train_neg["label"] = "__label__0" # Need the __class__ prefix for fast-text model specs
df_train_pos["label"] = "__label__1"
df_train = pd.concat([df_train_pos,df_train_neg])
df_train

C:\Users\Victor\AppData\Local\Temp\ipykernel_32428\481088145.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_train_neg = pd.read_csv("data/twitter-datasets/train_neg.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
C:\Users\Victor\AppData\Local\Temp\ipykernel_32428\481088145.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_train_pos = pd.read_csv("data/twitter-datasets/train_pos.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)


,tweets,label
0,<user> i dunno justin read my mention or not ....,__label__1
1,"because your logic is so dumb , i won't even c...",__label__1
2,<user> just put casper in a box ! looved the...,__label__1
3,<user> <user> thanks sir > > don't trip lil ma...,__label__1
4,visiting my brother tmr is the bestest birthda...,__label__1
...,...,...
99063,can't wait to fake tan tonight ! hate being pale,__label__0
99064,<user> darling i lost my internet connection ....,__label__0
99065,kanguru defender basic 4 gb usb 2.0 flash driv...,__label__0
99066,rizan is sad now,__label__0


In [163]:
df_test = pd.read_csv("data/twitter-datasets/test_data.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
#df_test["tweets"] = df_test["tweets"].apply(lambda row: row.split(",",2)[1])

C:\Users\Victor\AppData\Local\Temp\ipykernel_32428\3045235564.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_test = pd.read_csv("data/twitter-datasets/test_data.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)


In [164]:
df_test

,tweets
0,"1,sea doo pro sea scooter ( sports with the po..."
1,"2,<user> shucks well i work all week so now i ..."
2,"3,i cant stay away from bug thats my baby"
3,"4,<user> no ma'am ! ! ! lol im perfectly fine ..."
4,"5,whenever i fall asleep watching the tv , i a..."
...,...
9995,"9996,had a nice time w / my friend lastnite"
9996,"9997,<user> no it's not ! please stop !"
9997,"9998,not without my daughter ( dvd two-time os..."
9998,"9999,<user> have fun in class sweetcheeks"


In [165]:
def remove_tags(df):
    df_cleaned = df.copy()
    df_cleaned['tweets'] = df_cleaned['tweets'].apply(lambda tweet: re.sub(r'<.*?>', '', tweet).strip())
    return df_cleaned

def tokenize_and_preprocess(df, stop_words = False, stemming = False, lemmatization = False):
    df_cleaned = df.copy()
    df_cleaned['tokens'] = df_cleaned['tweets'].apply(lambda tweet: word_tokenize(tweet))
    # remove stop words
    if stop_words:
        stop_words = stopwords.words('english')
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])
    # stemming
    if stemming:
        ps = PorterStemmer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [ps.stem(token) for token in tokens])
    # lemmatization
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [wordnet_lemmatizer.lemmatize(token) for token in tokens])
    # remove the tweets columns
    df_cleaned.drop(['tweets'], axis=1, inplace=True)
    df_cleaned = df_cleaned.reindex(columns=['tokens', 'label'])
    #df_cleaned["tweets"] = df_cleaned["tokens"].apply(lambda tokens: " ".join(tokens))
    return df_cleaned

In [166]:
#df_train = tokenize_and_preprocess(remove_tags(df_train),stop_words=True,stemming=True,lemmatization=True)

In [167]:
df_train = remove_tags(df_train)

In [168]:
df_train

,tweets,label
0,i dunno justin read my mention or not . only j...,__label__1
1,"because your logic is so dumb , i won't even c...",__label__1
2,just put casper in a box ! looved the battle ...,__label__1
3,thanks sir > > don't trip lil mama ... just ke...,__label__1
4,visiting my brother tmr is the bestest birthda...,__label__1
...,...,...
99063,can't wait to fake tan tonight ! hate being pale,__label__0
99064,darling i lost my internet connection .. and i...,__label__0
99065,kanguru defender basic 4 gb usb 2.0 flash driv...,__label__0
99066,rizan is sad now,__label__0


In [169]:
#df_test = tokenize_and_preprocess(remove_tags(df_test),stop_words=True,stemming=True,lemmatization=True)
df_test = remove_tags(df_test)

In [170]:
df_test

,tweets
0,"1,sea doo pro sea scooter ( sports with the po..."
1,"2, shucks well i work all week so now i can't ..."
2,"3,i cant stay away from bug thats my baby"
3,"4, no ma'am ! ! ! lol im perfectly fine and no..."
4,"5,whenever i fall asleep watching the tv , i a..."
...,...
9995,"9996,had a nice time w / my friend lastnite"
9996,"9997, no it's not ! please stop !"
9997,"9998,not without my daughter ( dvd two-time os..."
9998,"9999, have fun in class sweetcheeks"


In [171]:
train_file = 'data/fasstext_train.txt'
df_train.to_csv(train_file, header=None, index=False, sep=' ', columns=["label","tweets"])
test_file =  'data/fasstext_test.txt'
df_test.to_csv(test_file, header=None, index=False, sep=' ', columns=["tweets"])

In [172]:
model = train_supervised(input=train_file)

In [173]:
print('vocab size: ', len(model.words))

vocab size:  140672


In [174]:
print('label size: ', len(model.labels))

label size:  2


In [175]:
predictions = []
file1 = open('data/fasstext_test.txt', 'r')
lines = file1.readlines()
for line in lines:
    pred = model.predict(line.strip())[0][0]
    if pred == "__label__0":
        predictions.append(-1)
    else:    
        predictions.append(1)

In [176]:
predictions

[-1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,


In [177]:
ids=[i for i in range(1,len(predictions)+1)]
with open("data/submission.csv", 'w', newline='') as csvfile:
    fieldnames = ['Id', 'Prediction']
    writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
    writer.writeheader()
    for r1, r2 in zip(ids, predictions):
        writer.writerow({'Id':int(r1),'Prediction':int(r2)})